In [1]:
import numpy as np
import NUTS as hmc
import hmc as hmc2
import matplotlib.pyplot as plt
import corner
import time
import sklearn.datasets as skld

In [2]:
plt.rcParams['figure.figsize'] = [10, 10]

limits = [(-40,40),(-40,40),(-40,40)]
nparam = len(limits)

cov = skld.make_spd_matrix(nparam)
#cov = np.eye(nparam)
inv_cov = np.linalg.inv(cov)
print(cov)


[[ 0.30230588 -0.31399687  0.07456456]
 [-0.31399687  3.33414847 -0.0582953 ]
 [ 0.07456456 -0.0582953   0.44376323]]


In [3]:
def mock_posterior_and_gradient(p):
    grad = -p @ inv_cov
    #j = theory_jacobian(p, *self.args).T
    logP = 0.5 * grad @ p.T
    #logP_jacobian = j @ dlogL_dCl
    #logP_jacobian = p * (-1)
    #mock_posterior_and_gradient.counter += 1
    return logP, grad

In [4]:
def run_hmc(n_it, hmc, cov, burnin):
    #rank = 5
    rank = nparam
    #np.random.seed(100 + rank)
    #C = np.eye(nparam)
    # mass matrix
    sampler = hmc.HMC(mock_posterior_and_gradient, cov, limits)
    # first sample starts at fid
    fid_params  = np.zeros(nparam)
    results = sampler.NUTS(fid_params, 0.5, n_it, burnin)

    return sampler

In [5]:
def run_hmc2(n_it, epsilon, hmc, spit, cov):
    #rank = 5
    rank = nparam
    #np.random.seed(100 + rank)
    #C = np.eye(nparam)
    # mass matrix
    sampler = hmc.HMC(mock_posterior_and_gradient, cov, epsilon, spit, limits)
    # first sample starts at fid
    fid_params  = np.zeros(nparam)
    results = sampler.sample(n_it, fid_params)

    return sampler

In [6]:
nit = 1000
burnin = 50

start = time.time()
chain = run_hmc(nit, hmc, cov, burnin)
print("Total time to run is %.2f s" %(time.time()-start))
print("")

start = time.time()
#chain2 = run_hmc(nit, hmc, cov, burnin)
chain2 = run_hmc2(nit, 0.02, hmc2, 10, cov)
print("Total time to run is %.2f s" %(time.time()-start))

0.5
Reasonable epsilon is 0.5
final epsilon is 1.2759082049541972
Step 105 of 1050
Step 210 of 1050
Step 315 of 1050
Step 420 of 1050
Step 525 of 1050
Step 630 of 1050
Step 735 of 1050
Step 840 of 1050
Step 945 of 1050
Total time to run is 0.53 s

Accept 0 alpha=1.00  p=0.55  ΔH=0.000
Accept 50 alpha=1.00  p=0.81  ΔH=-0.000
Accept 100 alpha=1.00  p=0.31  ΔH=0.000
Accept 150 alpha=1.00  p=0.18  ΔH=-0.000
Accept 200 alpha=1.00  p=0.75  ΔH=0.000
Accept 250 alpha=1.00  p=0.48  ΔH=0.000
Accept 300 alpha=1.00  p=0.99  ΔH=0.000
Accept 350 alpha=1.00  p=0.57  ΔH=-0.000
Accept 400 alpha=1.00  p=0.14  ΔH=-0.000
Accept 450 alpha=1.00  p=0.98  ΔH=0.000
Accept 500 alpha=1.00  p=0.78  ΔH=0.000
Accept 550 alpha=1.00  p=0.62  ΔH=0.000
Accept 600 alpha=1.00  p=0.15  ΔH=-0.000
Accept 650 alpha=1.00  p=0.12  ΔH=-0.000
Accept 700 alpha=1.00  p=0.36  ΔH=-0.000
Accept 750 alpha=1.00  p=0.01  ΔH=-0.000
Accept 800 alpha=1.00  p=0.41  ΔH=-0.000
Accept 850 alpha=1.00  p=0.74  ΔH=-0.000
Accept 900 alpha=1.00  p=

In [7]:
#paths = np.array(chain.paths)
#anti_paths = np.array(chain.anti_paths)

trace = np.array(chain.trace)
calls = np.array(chain.ncall_list)

#print(trace)

print(trace.shape)
print(calls.shape)

trace2 = np.array(chain2.trace)
calls2 = np.array(chain2.ncall_list)
print(trace2.shape)
print(calls2.shape)

(0,)
(0,)
(1000, 3)
(1000,)


In [8]:
#plt.scatter(paths[:,0],paths[:,1],s=3)
#plt.scatter(anti_paths[:,0],anti_paths[:,1],s=3)
#plt.show()

plt.scatter(trace[:,0],trace[:,1],s=5)
#plt.scatter(trace2[:,0],trace2[:,1],s=5)
plt.show()

figure = corner.corner(trace)
figure2 = corner.corner(trace2)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
def plot_conv(trace,dim,botlim,toplim, hmc, cov, calls):
    STDs, means = [], []
    for i in range(trace.shape[0]):
        STD=np.std(trace[:i,dim])
        STDs.append(STD/np.sqrt(cov[dim,dim]))
        means.append(np.mean(trace[:i,dim]))
    plt.plot(calls, STDs, label="STD/sigma hmc %s" %hmc)
    plt.plot(calls, means, label="mean hmc %s" %hmc)
    plt.hlines([0,1],0,calls[-1],ls=":", color="k")
    plt.legend()
    plt.title("Mean & STD vs L-calls for dim %.0f" %dim)
    #plt.ylim(botlim,toplim)
    
    #print(np.std(trace[:,dim]))
    

In [ ]:
print(np.sqrt(cov))

plot_conv(trace,1,-1,1.5,1, cov, calls)
plot_conv(trace2,1,-1,1.5,2, cov, calls2)
plt.show()